In [36]:
import paramiko
from scp import SCPClient

import datetime
import subprocess
import shlex

import requests

import uuid

import sys
sys.path.append('..')
import usaxs_utils

In [7]:
ip = '3.147.54.9'


# Define SSH connection parameters
hostname = '3.147.54.9'  # Replace with your server address
port = 22  # Default SSH port
username = 'ec2-user'  # Replace with your SSH username
pem_file_path = '/home/bgpelkie/silica-np-opt-key.pem'  # Path to your .pem private key file
local_file = 'usaxs_queue.txt'  # Path to the local file you want to copy
remote_path = '~/usaxs_queue/usaxs_queue.txt'  # Path where the file will be copied on the server

# Establish SSH connection using PEM file for authentication
try:
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())  # Automatically add unknown host keys

    # Load the private key
    private_key = paramiko.RSAKey.from_private_key_file(pem_file_path)

    print('connecting with ssh')
    # Connect using the private key for authentication
    ssh.connect(hostname, port=port, username=username, pkey=private_key)
    print('copying file with scp')
    # Copy the file using SCP
    with SCPClient(ssh.get_transport()) as scp:
        scp.put(local_file, remote_path)  # Upload the local file to the remote server

    print(f"File {local_file} successfully copied to {hostname}:{remote_path}")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    ssh.close()


connecting with ssh
copying file with scp
File usaxs_queue.txt successfully copied to 3.147.54.9:~/usaxs_queue/usaxs_queue.txt


In [10]:


try:
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())  # Automatically add unknown host keys

    # Load the private key
    private_key = paramiko.RSAKey.from_private_key_file(pem_file_path)

    # Connect using the private key for authentication
    ssh.connect(hostname, port=port, username=username, pkey=private_key)

    # Copy the file using SCP (from remote to local)
    with SCPClient(ssh.get_transport()) as scp:
        scp.get(remote_path, local_file)  # Download the remote file to the local machine

    print(f"File {remote_path} successfully copied to {local_file}")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    ssh.close()

File ~/usaxs_queue/usaxs_queue.txt successfully copied to usaxs_queue.txt


In [19]:
def copy_remote_file(instance_ip, user_name, remote_path, local_path, pem_path,copy_from=True):
    """
    Copies a remote file to the local machine using scp.

    Args:
        instance_ip (str): The public IP address or public DNS of the EC2 instance.
        user_name (str): The username for connecting to the EC2 instance (e.g., 'ec2-user' or 'ubuntu').
        remote_path (str): The full path to the file on the EC2 instance.
        local_path (str): The full path where the file should be saved locally.
        pem_path (str): The path to the .pem file used for SSH authentication.
    """
    # print(instance_ip)
    # print(user_name)
    # print(remote_path)
    # print(local_path)
    # print(pem_path)
    try:
        command = [
            "scp",
            "-i",
            str(pem_path),
        ]
        if copy_from:
            command.extend([
                f"{user_name}@{instance_ip}:{remote_path}",
                str(local_path),
            ])
        else:
            command.extend([
                str(local_path),
                f"{user_name}@{instance_ip}:{remote_path}",
            ])
        
        subprocess.run(shlex.split(' '.join(command)), check=True, capture_output=True)
        print(f"File copied successfully from {remote_path} to {local_path}")
    
    except subprocess.CalledProcessError as e:
         print(f"Error copying file: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [15]:
sample_name = 'remotetest'
sample_uuid = 'aaaabbbbcccc'
sample_composition = '1'

In [25]:
## update remote file to trigger new measurement

ip = '18.216.44.137'
remote_username = 'ubuntu'
pem_path = '/home/bgpelkie/silica-np-opt-key.pem'
date_format = "%Y-%m-%d %H:%M:%S"


local_working_directory = './usaxs_queue'
remote_filepath = '~/usaxs_queue/'
data_package_filename = 'packet.txt'

In [16]:
trigger_data = [datetime.datetime.now().strftime(date_format), sample_name, sample_composition, sample_uuid]

In [18]:
with open(f'{local_working_directory}/{data_package_filename}', 'wt') as f:
    for entry in trigger_data:
        f.write(entry + '\n')

In [26]:
copy_remote_file(ip, remote_username, remote_filepath, f'{local_working_directory}/{data_package_filename}', pem_file_path, copy_from=False)

File copied successfully from ~/usaxs_queue/ to ./usaxs_queue/packet.txt


## Test AFL rinse trigger

In [39]:
sampleid = str(uuid.uuid4())

In [40]:
sampleid

'aede8428-8897-4def-a6e5-087cb2c85b21'

In [41]:
ip = '127.0.0.1'
url = f'http://{ip}:5000/check_usaxs_status'
data = {'id':sampleid}

In [42]:
r = requests.post(url, json = data)

In [43]:
r

<Response [200]>

In [44]:
r.content

b'{\n  "data_files": [\n    "USAXS-Precursor_background_noTEOS-1.csv",\n    "USAXS-test-2.csv",\n    "TEsting_emptydata_aede8428-8897-4def-a6e5-087cb2c85b21.txt",\n    "USAXS-Ethanol_background_2-Ethanol.csv",\n    "USAXS-Control2-3.csv",\n    "WAXS-Control2-3.csv",\n    "WAXS-Ethanol_background_2-Ethanol.csv",\n    "USAXS-Control_3-3.csv",\n    "WAXS-test-2.csv",\n    "WAXS-Control1-2.csv",\n    "USAXS-Control1-2.csv",\n    "WAXS-Ethanol_background-Ethanol.csv",\n    "WAXS-Control_3-3.csv",\n    "USAXS-Ethanol_background-Ethanol.csv",\n    "WAXS-Precursor_background_noTEOS-1.csv"\n  ],\n  "usaxs_status": "complete"\n}\n'